In [1]:
# Import our dependencies
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf



**Read the data - information from Kaggle**

In [2]:
# Specify the file path
file_path = r'C:\Users\aumek\OneDrive\Desktop\Git_Personnal_Rep\Resource_mustang\merged_data.csv'

# Read the CSV file into pandas
df = pd.read_csv(file_path)
df.head(5)


C:\Users\aumek\AppData\Local\Temp\ipykernel_27744\1200018108.py:5: DtypeWarning: Columns (11,19,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Unnamed: 0,rid,course,time,date,title,rclass,band,ages,distance,...,OR,father,mother,gfather,runners,margin,weight,res_win,res_place,price
0,0,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Pour Moi,Laughing Water,Duke Of Marmalade,9.0,1.199095,66.0,1.0,1.0,NaN
1,1,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,110.0,Getaway,Shouette,Sadler's Wells,9.0,1.199095,65.0,0.0,1.0,NaN
2,2,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Vale Of York,Finnmark,Halling,9.0,1.199095,72.0,0.0,1.0,NaN
3,3,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Sixties Icon,Highland Jig,Norse Dancer,9.0,1.199095,72.0,0.0,0.0,NaN
4,4,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Iffraaj,Hallowed Park,Barathea,9.0,1.199095,72.0,0.0,0.0,NaN


In [3]:
 # Get a list of all of our columns for easy reference
df.columns

Index(['Unnamed: 0', 'rid', 'course', 'time', 'date', 'title', 'rclass',
       'band', 'ages', 'distance', 'condition', 'hurdles', 'prizes',
       'winningTime', 'prize', 'metric', 'countryCode', 'ncond', 'class',
       'currency', 'horseName', 'age', 'saddle', 'decimalPrice', 'isFav',
       'trainerName', 'jockeyName', 'position', 'positionL', 'dist',
       'weightSt', 'weightLb', 'overWeight', 'outHandicap', 'headGear', 'RPR',
       'TR', 'OR', 'father', 'mother', 'gfather', 'runners', 'margin',
       'weight', 'res_win', 'res_place', 'price'],
      dtype='object')

**Preparing the data For Model1 - where we are predicting if the 'favorite' horse wins, we used two datasets from "Kaggle" as listed below:**

*dataset 1 = races_* columns description: 
- rid - Race id; 
- Distance
- winningTime - Best time shown
- metric - Distance in meters
- class - class type (created from rclass feature).

*dataset 2 = horses_* columns description: 
- rid - Race id (column used to merge the 2 datasets)
- age - Horse age; 
- saddle - Saddle # where horse starts; 
- isFav - Was horse favorite before start? Can be more then one fav in a race;  
- position - Finishing position, 40 if horse didn't finish; 
- positionL - how far a horse has finished from the pursued horse, horses corpses; 
- outHandicap - Handicap; 
- RPR - RP Rating; 
- TR - Topspeed; 
- OR - Official Rating father - Horse's Father name; 
- runners - Runners total; 
- weight - Horse weight in kg; 
- res_win - Horse won or not; 


**Data Cleanup**

In [6]:
# Drop columns
clean_df = df.loc[:, [ 'rid', 'winningTime', 'metric', 'ncond', 'class',
       'age', 'decimalPrice', 'isFav','position', 'positionL', 'distance','outHandicap', 'RPR','saddle',
       'TR', 'OR', 'runners', 'weight', 'res_win']]
clean_df.head(5)

,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4.0,0.076923,0.0,1.0,NaN,2m,NaN,103.0,9.0,103.0,NaN,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6.0,0.444444,1.0,2.0,15,2m,NaN,96.0,7.0,89.0,110.0,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7.0,0.019608,0.0,3.0,5.5,2m,NaN,97.0,5.0,88.0,NaN,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6.0,0.230769,0.0,4.0,10,2m,NaN,89.0,4.0,75.0,NaN,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7.0,0.038462,0.0,5.0,5.5,2m,NaN,82.0,6.0,67.0,NaN,9.0,72.0,0.0


In [7]:
import re

#remove special characters and normalize distance to show all as 'meters'

# Convert 'dist' column to string data type
clean_df['distance'] = clean_df['distance'].astype(str)

# Define the function to clean the distance values
def clean_distance(distance):
    if re.search(r'\d+m', distance):
        meters = re.findall(r'\d+', distance)
        return int(meters[0])
    elif re.search(r'\d+f', distance):
        furlongs = re.findall(r'\d+', distance)
        meters = int(furlongs[0]) * 201
        return meters
    elif re.search(r'\dm\d+f', distance):
        meters = re.findall(r'\d+m', distance)
        furlongs = re.findall(r'\d+f', distance)
        meters = int(meters[0].strip('m'))
        furlongs = int(furlongs[0].strip('f')) * 201
        return meters + furlongs
    else:
        return distance

# Apply the clean_distance function to the 'dist' column
clean_df['distance'] = clean_df['distance'].apply(clean_distance)

# Print the cleaned DataFrame
clean_df.head(5)


,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4.0,0.076923,0.0,1.0,NaN,2,NaN,103.0,9.0,103.0,NaN,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6.0,0.444444,1.0,2.0,15,2,NaN,96.0,7.0,89.0,110.0,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7.0,0.019608,0.0,3.0,5.5,2,NaN,97.0,5.0,88.0,NaN,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6.0,0.230769,0.0,4.0,10,2,NaN,89.0,4.0,75.0,NaN,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7.0,0.038462,0.0,5.0,5.5,2,NaN,82.0,6.0,67.0,NaN,9.0,72.0,0.0


In [8]:
#to verify the data type
clean_df.dtypes

rid               int64
winningTime     float64
metric          float64
ncond             int64
class             int64
age             float64
decimalPrice    float64
isFav           float64
position        float64
positionL        object
distance         object
outHandicap     float64
RPR             float64
saddle          float64
TR              float64
OR              float64
runners         float64
weight          float64
res_win         float64
dtype: object

In [9]:
#changing datatype for 'positionL' and 'dist' to numeric
clean_df['positionL'] = pd.to_numeric(clean_df['positionL'], errors='coerce')
clean_df['distance'] = pd.to_numeric(clean_df['distance'], errors='coerce')

# Verify the updated data types
print(clean_df.dtypes)


rid               int64
winningTime     float64
metric          float64
ncond             int64
class             int64
age             float64
decimalPrice    float64
isFav           float64
position        float64
positionL       float64
distance        float64
outHandicap     float64
RPR             float64
saddle          float64
TR              float64
OR              float64
runners         float64
weight          float64
res_win         float64
dtype: object


In [10]:
# Specify the columns you want to drop NA values
columns_to_drop_na = ['isFav', 'res_win']

# Specify the values representing blanks in those columns
blank_values = ['', ' ', '-']

# Replace specific blank values with NaN
clean_df.replace(blank_values, np.nan, inplace=True)

# Drop rows with NA values in the specified columns
clean_df.dropna(subset=columns_to_drop_na, inplace=True)

# Reset the index
clean_df.reset_index(drop=True, inplace=True)


In [11]:
# save to csv to visualize data
clean_df.to_csv('Resources/clean_data_model1_v2.csv', index=False)


In [12]:
# Count the remaining rows and columns in the DataFrame
num_rows, num_columns = clean_df.shape

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)



Number of rows: 149513
Number of columns: 19


In [ ]:
y = clean_df["isFav"]
X = clean_df.drop(columns="isFav")

In [ ]:
 from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

In [ ]:
 from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
coverted_df= pd.get_dummies(clean_df)
coverted_df

In [ ]:

# Split our preprocessed data into our features and target arrays
#from sklearn.model_selection import train_test_split
sample_size = int(len(coverted_df) * 0.5)  # 50% of the original dataset as I run out of RAM
sample_df = coverted_df[:sample_size]


# Split the preprocessed data into features and target arrays
X = sample_df.drop('isFav', axis=1).values
y = sample_df['isFav'].values


# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)